# B1.iii - Extreme Gradient Boosting (XGBoost) με Word2Vec

Αυτό το notebook υλοποιεί τον αλγόριθμο **XGBoost** (Extreme Gradient Boosting) σε συνδυασμό με **Word2Vec** για την ταξινόμηση νομικών κειμένων. Βασίζεται στον κώδικα από το αρχείο `main_XgBoost.py`.

## Περιεχόμενα
1. [Τεχνικές Λεπτομέρειες XGBoost](#xgboost-technical)
2. [Παράμετροι XGBoost](#xgboost-parameters)
3. [Συνδυασμός XGBoost με Word2Vec](#xgboost-word2vec)
4. [Υλοποίηση και Εκτέλεση](#implementation)
5. [Σύγκριση με άλλους Αλγορίθμους](#comparison)

## 1. Τεχνικές Λεπτομέρειες XGBoost {#xgboost-technical}

Το **XGBoost** (Extreme Gradient Boosting) είναι μια υλοποίηση του gradient boosting που έχει σχεδιαστεί για ταχύτητα και απόδοση.

### Θεωρητικό Υπόβαθρο

Το XGBoost βασίζεται στην ιδέα του **ensemble learning**, όπου πολλά αδύναμα μοντέλα (weak learners) συνδυάζονται για να δημιουργήσουν ένα ισχυρό μοντέλο.

#### Gradient Boosting Framework

Η βασική ιδέα είναι η **sequential training** μοντέλων:

$$\hat{y}_i^{(t)} = \hat{y}_i^{(t-1)} + f_t(x_i)$$

όπου:
- $\hat{y}_i^{(t)}$ είναι η πρόβλεψη στο βήμα $t$
- $f_t(x_i)$ είναι το νέο δέντρο που προστίθεται

#### Objective Function

Το XGBoost ελαχιστοποιεί την εξής objective function:

$$\mathcal{L}^{(t)} = \sum_{i=1}^{n} l(y_i, \hat{y}_i^{(t-1)} + f_t(x_i)) + \Omega(f_t)$$

όπου:
- $l$ είναι η loss function
- $\Omega(f_t)$ είναι ο regularization term

### Βασικά Χαρακτηριστικά

1. **Regularization**: Το XGBoost περιλαμβάνει L1 και L2 regularization
2. **Tree Pruning**: Χρησιμοποιεί post-pruning με το max_depth parameter
3. **Parallel Processing**: Παραλληλοποίηση της κατασκευής δέντρων
4. **Built-in Cross-Validation**: Ενσωματωμένη υποστήριξη για CV
5. **Missing Value Handling**: Αυτόματος χειρισμός missing values

## 2. Παράμετροι XGBoost {#xgboost-parameters}

### Κύριες Παράμετροι

| Παράμετρος | Περιγραφή | Τιμή στο Project |
|------------|-----------|------------------|
| `n_estimators` | Αριθμός δέντρων (boosting rounds) | 100 |
| `learning_rate` | Ρυθμός μάθησης (step size shrinkage) | 0.1 |
| `max_depth` | Μέγιστο βάθος δέντρων | 3 |
| `eval_metric` | Μετρική αξιολόγησης | 'mlogloss' |
| `random_state` | Seed για reproducibility | 42 |

### Αναλυτική Περιγραφή Παραμέτρων

#### 1. n_estimators (Αριθμός Δέντρων)
- **Ρόλος**: Καθορίζει πόσα δέντρα θα κατασκευαστούν
- **Επίδραση**: 
  - Περισσότερα δέντρα → καλύτερη απόδοση, αλλά κίνδυνος overfitting
  - Λιγότερα δέντρα → ταχύτερη εκπαίδευση, αλλά κίνδυνος underfitting
- **Συνήθεις Τιμές**: 50-1000+

#### 2. learning_rate (Ρυθμός Μάθησης)
- **Ρόλος**: Ελέγχει τη συνεισφορά κάθε δέντρου
- **Μαθηματική Έκφραση**: $\hat{y}_i^{(t)} = \hat{y}_i^{(t-1)} + \eta \cdot f_t(x_i)$
- **Επίδραση**:
  - Μικρότερη τιμή → πιο σταθερή σύγκλιση, χρειάζεται περισσότερα δέντρα
  - Μεγαλύτερη τιμή → ταχύτερη εκπαίδευση, κίνδυνος overshooting
- **Συνήθεις Τιμές**: 0.01-0.3

#### 3. max_depth (Μέγιστο Βάθος)
- **Ρόλος**: Ελέγχει την πολυπλοκότητα κάθε δέντρου
- **Επίδραση**:
  - Μεγαλύτερο βάθος → πιο πολύπλοκα μοντέλα, κίνδυνος overfitting
  - Μικρότερο βάθος → απλούστερα μοντέλα, κίνδυνος underfitting
- **Συνήθεις Τιμές**: 3-10

#### 4. eval_metric (Μετρική Αξιολόγησης)
- **'mlogloss'**: Multi-class logarithmic loss
- **Μαθηματική Έκφραση**: $-\frac{1}{N} \sum_{i=1}^{N} \sum_{j=1}^{M} y_{i,j} \log(p_{i,j})$
- **Χρήση**: Κατάλληλη για πολυκλασικά προβλήματα

## 3. Συνδυασμός XGBoost με Word2Vec {#xgboost-word2vec}

### Γιατί XGBoost + Word2Vec;

1. **Πλούσια Αναπαράσταση**: Το Word2Vec δημιουργεί dense vectors που κωδικοποιούν σημασιολογικές σχέσεις
2. **Non-linear Patterns**: Το XGBoost μπορεί να ανακαλύψει πολύπλοκες σχέσεις στα Word2Vec features
3. **Feature Importance**: Το XGBoost παρέχει interpretability μέσω feature importance
4. **Robust Performance**: Καλή απόδοση σε διάφορα domains

### Διαδικασία Επεξεργασίας

```
Κείμενο → Tokenization → Word2Vec Training → Document Vectors → XGBoost Training
```

#### Document Vectorization Strategies

1. **Average Pooling**: $\vec{d} = \frac{1}{|d|} \sum_{w \in d} \vec{w}$
2. **Weighted Average**: $\vec{d} = \frac{\sum_{w \in d} tf(w) \cdot \vec{w}}{\sum_{w \in d} tf(w)}$
3. **TF-IDF Weighted**: $\vec{d} = \frac{\sum_{w \in d} tfidf(w) \cdot \vec{w}}{\sum_{w \in d} tfidf(w)}$

### Πλεονεκτήματα του Συνδυασμού

| Χαρακτηριστικό | Word2Vec | XGBoost | Συνδυασμός |
|----------------|----------|---------|------------|
| Σημασιολογική Αναπαράσταση | ✓ | - | ✓ |
| Non-linear Learning | - | ✓ | ✓ |
| Feature Interactions | - | ✓ | ✓ |
| Interpretability | - | ✓ | ✓ |
| Regularization | - | ✓ | ✓ |
| Scalability | ✓ | ✓ | ✓ |

## 4. Υλοποίηση και Εκτέλεση {#implementation}

### Βήμα 1: Import Libraries και Setup

In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import warnings
warnings.filterwarnings('ignore')

# Εισαγωγή XGBoost
try:
    import xgboost as xgb
    print(f"XGBoost version: {xgb.__version__}")
except ImportError:
    print("XGBoost is not installed. Installing...")
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "xgboost"])
    import xgboost as xgb
    print(f"XGBoost version: {xgb.__version__}")

# Εισαγωγή Gensim για Word2Vec
try:
    from gensim.models import Word2Vec
    print(f"Gensim available for Word2Vec")
except ImportError:
    print("Gensim is not installed. Installing...")
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "gensim"])
    from gensim.models import Word2Vec
    print(f"Gensim available for Word2Vec")

# Προσπάθεια εισαγωγής utils
try:
    from utils import (
        load_and_preprocess_data,
        run_experiment,
        script_execution_timer
    )
    print("Utils module imported successfully")
    UTILS_AVAILABLE = True
except ImportError:
    print("Utils module not available. Will use simplified implementation.")
    UTILS_AVAILABLE = False

# Ρυθμίσεις για matplotlib
plt.style.use('default')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

print("\n=== XGBoost με Word2Vec για Ταξινόμηση Νομικών Κειμένων ===")
print("Jupyter Notebook Implementation")

### Βήμα 2: Παράμετροι και Configuration

In [ ]:
# =============================================================================
# ΠΑΡΑΜΕΤΡΟΙ CONFIGURATION
# =============================================================================

# Dataset Configuration
DATASET_CONFIG = "subject"  # available options: "volume", "chapter", "subject"
SUBSET_PERCENTAGE = 0.1     # 1.0 for full data, < 1.0 for subset (για γρήγορη εκτέλεση)
RANDOM_STATE = 42
TEST_SIZE = 0.2

# Word2Vec Parameters
WORD2VEC_VECTOR_SIZE = 100  # Διάσταση των word vectors
WORD2VEC_WINDOW = 5         # Context window size
WORD2VEC_MIN_COUNT = 2      # Minimum word frequency
WORD2VEC_WORKERS = 4        # Number of threads
WORD2VEC_SG = 0            # 0 = CBOW, 1 = Skip-gram
WORD2VEC_EPOCHS = 10       # Training epochs

# XGBoost Parameters
XGB_N_ESTIMATORS = 100     # Αριθμός δέντρων
XGB_LEARNING_RATE = 0.1    # Ρυθμός μάθησης
XGB_MAX_DEPTH = 3          # Μέγιστο βάθος δέντρων
XGB_EVAL_METRIC = 'mlogloss'  # Μετρική αξιολόγησης

print("Παράμετροι Configuration:")
print(f"Dataset: {DATASET_CONFIG}")
print(f"Subset percentage: {SUBSET_PERCENTAGE*100:.0f}%")
print(f"Word2Vec vector size: {WORD2VEC_VECTOR_SIZE}")
print(f"Word2Vec algorithm: {'CBOW' if WORD2VEC_SG == 0 else 'Skip-gram'}")
print(f"XGBoost estimators: {XGB_N_ESTIMATORS}")
print(f"XGBoost learning rate: {XGB_LEARNING_RATE}")
print(f"XGBoost max depth: {XGB_MAX_DEPTH}")

### Βήμα 3: Δημιουργία Απλουστευμένων Συναρτήσεων (Fallback)

In [ ]:
# =============================================================================
# SIMPLIFIED FUNCTIONS (αν δεν είναι διαθέσιμο το utils module)
# =============================================================================

import re
import string
from collections import Counter

def simple_tokenize(text):
    """Απλή tokenization για ελληνικά κείμενα"""
    if not isinstance(text, str):
        return []
    
    # Μετατροπή σε πεζά
    text = text.lower()
    
    # Αφαίρεση σημείων στίξης
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Διαχωρισμός σε λέξεις
    words = text.split()
    
    # Φιλτράρισμα κενών strings
    words = [word for word in words if word.strip()]
    
    return words

def create_sample_data():
    """Δημιουργία sample δεδομένων για testing"""
    print("Creating sample Greek legal documents for testing...")
    
    # Sample ελληνικά νομικά κείμενα
    sample_texts = [
        "Το άρθρο 1 του Συντάγματος ορίζει την Ελλάδα ως δημοκρατικό κράτος",
        "Σύμφωνα με τον Αστικό Κώδικα η συμφωνία δεσμεύει τα συμβαλλόμενα μέρη",
        "Ο Ποινικός Κώδικας προβλέπει ποινές για τα αδικήματα κατά της ζωής",
        "Το δικαστήριο αποφάσισε για την εφαρμογή των διατάξεων του νόμου",
        "Η Ευρωπαϊκή Σύμβαση Δικαιωμάτων του Ανθρώπου κατοχυρώνει βασικά δικαιώματα",
        "Το άρθρο 8 της ΕΣΔΑ προστατεύει το δικαίωμα στην ιδιωτική ζωή",
        "Η διαδικασία του άρθρου 867 του ΚΠολΔ αφορά την αναγνώριση αποφάσεων",
        "Σύμφωνα με το άρθρο 281 ΑΚ η αποζημίωση καλύπτει τη θετική και αρνητική ζημία"
    ]
    
    # Sample labels
    sample_labels = [
        "Συνταγματικό Δίκαιο",
        "Αστικό Δίκαιο", 
        "Ποινικό Δίκαιο",
        "Δικονομία",
        "Ευρωπαϊκό Δίκαιο",
        "Ευρωπαϊκό Δίκαιο",
        "Δικονομία",
        "Αστικό Δίκαιο"
    ]
    
    # Δημιουργία περισσότερων samples με variations
    extended_texts = []
    extended_labels = []
    
    for i in range(20):  # Δημιουργία 20 samples
        idx = i % len(sample_texts)
        text = sample_texts[idx]
        label = sample_labels[idx]
        
        # Προσθήκη μικρών variations
        if i >= len(sample_texts):
            text = f"{text} Επιπλέον διατάξεις του νόμου αριθμός {i}."
        
        extended_texts.append(text)
        extended_labels.append(label)
    
    return extended_texts, extended_labels

def document_to_vector(doc_words, w2v_model, vector_size):
    """Μετατροπή εγγράφου σε vector χρησιμοποιώντας Word2Vec"""
    vectors = []
    for word in doc_words:
        if word in w2v_model.wv:
            vectors.append(w2v_model.wv[word])
    
    if vectors:
        # Average pooling
        return np.mean(vectors, axis=0)
    else:
        # Return zero vector if no words found
        return np.zeros(vector_size)

print("Simplified functions created successfully")

### Βήμα 4: Φόρτωση και Προετοιμασία Δεδομένων

In [ ]:
# =============================================================================
# ΦΟΡΤΩΣΗ ΚΑΙ ΠΡΟΕΤΟΙΜΑΣΙΑ ΔΕΔΟΜΕΝΩΝ
# =============================================================================

if UTILS_AVAILABLE:
    print("Loading data using utils module...")
    try:
        texts_proc, labels_proc, unique_labels_proc, unique_labels_proc_str = load_and_preprocess_data(
            DATASET_CONFIG, SUBSET_PERCENTAGE, RANDOM_STATE
        )
        
        if texts_proc is None or len(texts_proc) == 0:
            print("Failed to load data from utils. Using sample data.")
            texts_proc, labels_str = create_sample_data()
            # Convert labels to numeric
            unique_labels_str = list(set(labels_str))
            label_to_idx = {label: idx for idx, label in enumerate(unique_labels_str)}
            labels_proc = [label_to_idx[label] for label in labels_str]
            unique_labels_proc = list(range(len(unique_labels_str)))
            unique_labels_proc_str = unique_labels_str
        else:
            print(f"Successfully loaded {len(texts_proc)} documents")
            print(f"Number of unique labels: {len(unique_labels_proc)}")
            print(f"Labels: {unique_labels_proc_str}")
            
    except Exception as e:
        print(f"Error loading data with utils: {e}")
        print("Using sample data instead.")
        texts_proc, labels_str = create_sample_data()
        unique_labels_str = list(set(labels_str))
        label_to_idx = {label: idx for idx, label in enumerate(unique_labels_str)}
        labels_proc = [label_to_idx[label] for label in labels_str]
        unique_labels_proc = list(range(len(unique_labels_str)))
        unique_labels_proc_str = unique_labels_str
else:
    print("Utils not available. Using sample data...")
    texts_proc, labels_str = create_sample_data()
    
    # Convert string labels to numeric
    unique_labels_str = list(set(labels_str))
    label_to_idx = {label: idx for idx, label in enumerate(unique_labels_str)}
    labels_proc = [label_to_idx[label] for label in labels_str]
    unique_labels_proc = list(range(len(unique_labels_str)))
    unique_labels_proc_str = unique_labels_str

print(f"\nDataset Statistics:")
print(f"Number of documents: {len(texts_proc)}")
print(f"Number of classes: {len(unique_labels_proc)}")
print(f"Class names: {unique_labels_proc_str}")

# Έλεγχος κατανομής κλάσεων
label_counts = Counter(labels_proc)
print(f"\nClass distribution:")
for label_idx, count in label_counts.items():
    label_name = unique_labels_proc_str[label_idx]
    print(f"  {label_name}: {count} documents")

### Βήμα 5: Tokenization και Word2Vec Training

In [ ]:
# =============================================================================
# TOKENIZATION ΚΑΙ WORD2VEC TRAINING
# =============================================================================

print("Starting tokenization...")

# Tokenization των κειμένων
tokenized_texts = []
for text in texts_proc:
    tokens = simple_tokenize(text)
    tokenized_texts.append(tokens)

print(f"Tokenized {len(tokenized_texts)} documents")

# Εμφάνιση στατιστικών tokenization
token_counts = [len(tokens) for tokens in tokenized_texts]
print(f"Average tokens per document: {np.mean(token_counts):.1f}")
print(f"Min tokens: {np.min(token_counts)}")
print(f"Max tokens: {np.max(token_counts)}")

# Δείγμα tokenized text
print(f"\nSample tokenized text:")
print(f"Original: {texts_proc[0][:100]}...")
print(f"Tokenized: {tokenized_texts[0][:15]}...")

print("\nTraining Word2Vec model...")

# Εκπαίδευση Word2Vec μοντέλου
w2v_model = Word2Vec(
    sentences=tokenized_texts,
    vector_size=WORD2VEC_VECTOR_SIZE,
    window=WORD2VEC_WINDOW,
    min_count=WORD2VEC_MIN_COUNT,
    workers=WORD2VEC_WORKERS,
    sg=WORD2VEC_SG,
    epochs=WORD2VEC_EPOCHS,
    seed=RANDOM_STATE
)

print(f"Word2Vec model trained successfully!")
print(f"Vocabulary size: {len(w2v_model.wv)}")
print(f"Vector size: {w2v_model.vector_size}")

# Εμφάνιση sample words από το vocabulary
vocab_words = list(w2v_model.wv.index_to_key)[:10]
print(f"Sample vocabulary words: {vocab_words}")

### Βήμα 6: Δημιουργία Document Vectors

In [ ]:
# =============================================================================
# ΔΗΜΙΟΥΡΓΙΑ DOCUMENT VECTORS
# =============================================================================

print("Converting documents to vectors...")

# Μετατροπή κάθε εγγράφου σε vector
document_vectors = []
words_found_count = 0
total_words = 0

for doc_tokens in tokenized_texts:
    doc_vector = document_to_vector(doc_tokens, w2v_model, WORD2VEC_VECTOR_SIZE)
    document_vectors.append(doc_vector)
    
    # Στατιστικά για coverage
    found_in_vocab = sum(1 for word in doc_tokens if word in w2v_model.wv)
    words_found_count += found_in_vocab
    total_words += len(doc_tokens)

# Μετατροπή σε numpy array
X = np.array(document_vectors)
y = np.array(labels_proc)

print(f"Document vectors created successfully!")
print(f"Feature matrix shape: {X.shape}")
print(f"Labels shape: {y.shape}")
print(f"Vocabulary coverage: {words_found_count/total_words*100:.1f}%")

# Έλεγχος για NaN values
nan_count = np.isnan(X).sum()
if nan_count > 0:
    print(f"Warning: Found {nan_count} NaN values. Replacing with zeros.")
    X = np.nan_to_num(X)

print(f"Feature vector statistics:")
print(f"Mean: {X.mean():.4f}")
print(f"Std: {X.std():.4f}")
print(f"Min: {X.min():.4f}")
print(f"Max: {X.max():.4f}")

### Βήμα 7: Train/Test Split και XGBoost Training

In [ ]:
# =============================================================================
# TRAIN/TEST SPLIT ΚΑΙ XGBOOST TRAINING
# =============================================================================

print("Splitting data into train/test sets...")

# Διαχωρισμός σε train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=TEST_SIZE, 
    random_state=RANDOM_STATE,
    stratify=y
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

print("\nTraining XGBoost model...")

# Δημιουργία και εκπαίδευση XGBoost μοντέλου
xgb_model = xgb.XGBClassifier(
    n_estimators=XGB_N_ESTIMATORS,
    learning_rate=XGB_LEARNING_RATE,
    max_depth=XGB_MAX_DEPTH,
    eval_metric=XGB_EVAL_METRIC,
    random_state=RANDOM_STATE,
    verbosity=1  # Μειωμένη verbosity για cleaner output
)

# Εκπαίδευση μοντέλου
xgb_model.fit(X_train, y_train)

print("XGBoost model trained successfully!")

# Προβλέψεις
print("Making predictions...")
y_pred_train = xgb_model.predict(X_train)
y_pred_test = xgb_model.predict(X_test)

# Υπολογισμός accuracy
train_accuracy = accuracy_score(y_train, y_pred_train)
test_accuracy = accuracy_score(y_test, y_pred_test)

print(f"\nResults:")
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Overfitting Gap: {train_accuracy - test_accuracy:.4f}")

### Βήμα 8: Αναλυτική Αξιολόγηση και Οπτικοποιήσεις

In [ ]:
# =============================================================================
# ΑΝΑΛΥΤΙΚΗ ΑΞΙΟΛΟΓΗΣΗ
# =============================================================================

print("=== Classification Report ===")
report = classification_report(
    y_test, y_pred_test, 
    target_names=unique_labels_proc_str,
    digits=4
)
print(report)

# Confusion Matrix
print("\n=== Confusion Matrix ===")
cm = confusion_matrix(y_test, y_pred_test)
print(cm)

# Οπτικοποίηση Confusion Matrix
plt.figure(figsize=(10, 8))
sns.heatmap(
    cm, 
    annot=True, 
    fmt='d', 
    cmap='Blues',
    xticklabels=unique_labels_proc_str,
    yticklabels=unique_labels_proc_str
)
plt.title('Confusion Matrix - XGBoost με Word2Vec')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Feature Importance
print("\n=== Feature Importance Analysis ===")
feature_importance = xgb_model.feature_importances_

# Top 20 σημαντικότερα features
top_features = np.argsort(feature_importance)[-20:]
top_importance = feature_importance[top_features]

plt.figure(figsize=(10, 6))
plt.barh(range(len(top_features)), top_importance)
plt.yticks(range(len(top_features)), [f'Feature {i}' for i in top_features])
plt.xlabel('Feature Importance')
plt.title('Top 20 Feature Importance - XGBoost')
plt.tight_layout()
plt.show()

print(f"Average feature importance: {feature_importance.mean():.6f}")
print(f"Max feature importance: {feature_importance.max():.6f}")
print(f"Min feature importance: {feature_importance.min():.6f}")

### Βήμα 9: Παραμετρική Ανάλυση και Insights

In [ ]:
# =============================================================================
# ΠΑΡΑΜΕΤΡΙΚΗ ΑΝΑΛΥΣΗ ΚΑΙ INSIGHTS
# =============================================================================

print("=== XGBoost Model Information ===")
print(f"Number of estimators: {xgb_model.n_estimators}")
print(f"Learning rate: {xgb_model.learning_rate}")
print(f"Max depth: {xgb_model.max_depth}")
print(f"Number of features: {xgb_model.n_features_in_}")
print(f"Number of classes: {xgb_model.n_classes_}")

print("\n=== Word2Vec Model Information ===")
print(f"Vector size: {w2v_model.vector_size}")
print(f"Window size: {w2v_model.window}")
print(f"Min count: {w2v_model.min_count}")
print(f"Algorithm: {'CBOW' if w2v_model.sg == 0 else 'Skip-gram'}")
print(f"Training epochs: {w2v_model.epochs}")
print(f"Vocabulary size: {len(w2v_model.wv)}")

# Ανάλυση predictions per class
print("\n=== Predictions per Class ===")
from sklearn.metrics import classification_report
report_dict = classification_report(
    y_test, y_pred_test, 
    target_names=unique_labels_proc_str,
    output_dict=True
)

for class_name in unique_labels_proc_str:
    metrics = report_dict[class_name]
    print(f"{class_name}:")
    print(f"  Precision: {metrics['precision']:.4f}")
    print(f"  Recall: {metrics['recall']:.4f}")
    print(f"  F1-score: {metrics['f1-score']:.4f}")
    print(f"  Support: {int(metrics['support'])}")

# Macro και Weighted averages
print(f"\nMacro avg F1-score: {report_dict['macro avg']['f1-score']:.4f}")
print(f"Weighted avg F1-score: {report_dict['weighted avg']['f1-score']:.4f}")

# Performance Visualization
classes = unique_labels_proc_str
precision_scores = [report_dict[cls]['precision'] for cls in classes]
recall_scores = [report_dict[cls]['recall'] for cls in classes]
f1_scores = [report_dict[cls]['f1-score'] for cls in classes]

x_pos = np.arange(len(classes))
width = 0.25

plt.figure(figsize=(12, 6))
plt.bar(x_pos - width, precision_scores, width, label='Precision', alpha=0.8)
plt.bar(x_pos, recall_scores, width, label='Recall', alpha=0.8)
plt.bar(x_pos + width, f1_scores, width, label='F1-score', alpha=0.8)

plt.xlabel('Classes')
plt.ylabel('Score')
plt.title('Performance Metrics per Class - XGBoost με Word2Vec')
plt.xticks(x_pos, classes, rotation=45, ha='right')
plt.legend()
plt.ylim(0, 1.0)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## 5. Σύγκριση με άλλους Αλγορίθμους {#comparison}

### Σύγκριση XGBoost vs άλλα μοντέλα

| Αλγόριθμος | Πλεονεκτήματα | Μειονεκτήματα | Κατάλληλος για |
|------------|---------------|---------------|----------------|
| **XGBoost** | • Υψηλή απόδοση<br>• Feature importance<br>• Regularization<br>• Χειρισμός missing values | • Πολυπλοκότητα παραμέτρων<br>• Χρόνος εκπαίδευσης | Structured data, competitions |
| **SVM** | • Solid theoretical foundation<br>• Kernel trick<br>• Good with high-dim data | • Slow σε μεγάλα datasets<br>• Sensitive σε scaling | High-dimensional, small datasets |
| **Logistic Regression** | • Interpretable<br>• Fast training<br>• Probabilistic output | • Linear assumptions<br>• Limited expressiveness | Linear separable data, baselines |

### Συνδυασμός με Word2Vec Features

| Χαρακτηριστικό | SVM + BoW/TF-IDF | LogReg + Word2Vec | XGBoost + Word2Vec |
|----------------|------------------|-------------------|-----------|
| **Semantic Understanding** | ❌ | ✅ | ✅ |
| **Feature Interactions** | ❌ | ❌ | ✅ |
| **Non-linearity** | ✅ (με kernels) | ❌ | ✅ |
| **Interpretability** | ⚠️ | ✅ | ⚠️ |
| **Training Speed** | ⚠️ | ✅ | ⚠️ |
| **Memory Usage** | ⚠️ | ✅ | ⚠️ |

### Τι κάνει τον XGBoost + Word2Vec αποτελεσματικό;

1. **Rich Feature Representation**: Το Word2Vec δημιουργεί πλούσια semantic features
2. **Non-linear Learning**: Το XGBoost μπορεί να μάθει πολύπλοκες σχέσεις
3. **Feature Interactions**: Αυτόματη ανακάλυψη interactions μεταξύ features
4. **Regularization**: Αποφυγή overfitting με built-in regularization
5. **Robustness**: Καλή απόδοση σε διάφορα domains και datasets

### Πρακτικές Συμβουλές

#### Για XGBoost:
- Ξεκινήστε με `learning_rate=0.1` και `max_depth=3-6`
- Χρησιμοποιήστε early stopping με validation set
- Δοκιμάστε διαφορετικές τιμές για `n_estimators`
- Ελέγξτε το feature importance για insights

#### Για Word2Vec:
- `vector_size=100-300` για τα περισσότερα tasks
- `window=5-10` ανάλογα με το domain
- Χρησιμοποιήστε CBOW για μικρά datasets, Skip-gram για μεγάλα
- Ελέγξτε το vocabulary coverage

#### Για Document Vectorization:
- Average pooling είναι απλό και αποτελεσματικό
- Δοκιμάστε TF-IDF weighted averaging
- Εξετάστε την προσθήκη statistical features (length, avg word freq, κλπ.)

## Συμπεράσματα

Το notebook αυτό παρουσίασε την υλοποίηση του **XGBoost με Word2Vec** για ταξινόμηση νομικών κειμένων. Τα κύρια σημεία:

### Τεχνικά Highlights:
1. **XGBoost Algorithm**: Gradient boosting με regularization και feature importance
2. **Word2Vec Integration**: Semantic word embeddings για πλούσια αναπαράσταση
3. **Document Vectorization**: Average pooling για μετατροπή σε document-level features
4. **Parameter Optimization**: Επεξήγηση και tuning των κύριων παραμέτρων

### Πλεονεκτήματα της Προσέγγισης:
- **Semantic Understanding**: Κατανόηση σημασιολογικών σχέσεων μέσω Word2Vec
- **Non-linear Learning**: Ανακάλυψη πολύπλοκων patterns με XGBoost
- **Feature Interpretability**: Feature importance analysis
- **Robustness**: Καλή απόδοση σε διάφορα domains

### Προτάσεις για Βελτίωση:
1. **Hyperparameter Tuning**: Grid search ή Bayesian optimization
2. **Feature Engineering**: Προσθήκη statistical ή domain-specific features
3. **Ensemble Methods**: Συνδυασμός με άλλα μοντέλα
4. **Cross-Validation**: Πιο robust αξιολόγηση με K-fold CV

Η συνδυασμένη χρήση XGBoost + Word2Vec αποτελεί μια ισχυρή προσέγγιση για text classification που συνδυάζει την σημασιολογική αναπαράσταση του Word2Vec με την ικανότητα του XGBoost να μαθαίνει πολύπλοκες σχέσεις.